<a href="https://colab.research.google.com/github/JDM-1609/Statistical-Process-Control-in-Injection-Machines/blob/main/SPC_Analysis_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXTRACCIÓN DE ESTADÍSTICOS INICIALES

**SEPARACIÓN MANUAL DE LAS SECCIONES DEL CSV**

El siguiente código separa en DataFrames diferentes cada una de las secciones del CSV (Info. Contextual, estadísticos y datos crudos) de manera manual, dado que se le debe especificar los indices en donde comienza y termina cada sección. Este código tiene como limitante que si la distribución de los datos del CSV cambia el codigo se rompe (se tendrían que especificar los nuevos indices de inicio y fin de cada sección)

In [39]:
import csv
from pathlib import Path
from io import StringIO
import pandas as pd

# OPCIONAL: para visualización de los datos en el DF

pd.set_option("display.max_columns", 20) # Controla cuántas columnas se muestran sin omisiones
pd.set_option("display.float_format", lambda v: f"{v:.4g}") # Muestra valores con hasta 4 cifras significativas

# Encoding de los archivos del ALS
ENCODING_ALS = "cp1250"


In [2]:
# Carga de datos
from google.colab import files

uploaded = files.upload()
RUTA_IN = next(iter(uploaded.keys()))
print("Archivo cargado:", RUTA_IN)


Saving DATA PRUEBA.csv to DATA PRUEBA.csv
Archivo cargado: DATA PRUEBA.csv


In [7]:
# Indices para start & end de cada sección
# SECCIONES
idx_info = 0           # Información contextual
idx_stats_start = 2    # Start Estadísticos
idx_stats_end = 21     # End Estadísticos
idx_raw_start = 23     # Datos crudos

__LABORATORIO DE PRUEBAS EN CÓDIGO__

In [67]:
def parse_datos_crudos(lines, idx_start: int) -> pd.DataFrame:
    """
    Toma el bloque de 'Datos crudos' desde idx_start hasta el final del archivo
    y lo convierte en DataFrame.
    """
    block = "\n".join(lines[idx_start:])  # desde idx_start hasta EOF

    df_raw = pd.read_csv(
        StringIO(block),
        sep=",",
        decimal=",",
        thousands=".",
        quotechar='"',
        encoding=ENCODING_ALS,
        engine="python",
    )

    return df_raw


In [54]:
print(df_stats.index)
len(df_stats.columns)
df_stats.iloc[1]
#print(df_stats.index.tolist()[:5])

Index(['Valor nominal', 'Tolerancia inferior', 'Tolerancia superior', 'Mínimo',
       'Máximo', 'xqq', 'Rq', 'sq', 'Sigma', 'Cp', 'Cps', 'Cpi', 'Cpd', 'LISx',
       'LIIx', 'LISs', 'LIIs', 'n'],
      dtype='object', name='Estadísticos')


,Tolerancia inferior
,15.64
t4012 [s],0.59
t4018 [s],-0.15
t4015 [s],6.06
V4062 [cmł],1343
p4072 [bar],9.48
V4065 [cmł],NaN


In [6]:
#_______________________________________________________________________________________________

In [64]:
# FUNCIONES EMPLEADAS

# Lee el CSV como strings y devuelve una lista donde cada elemento es una línea del CSV
def leer_lineas(ruta_csv: str, encoding: str = ENCODING_ALS):
    raw = Path(ruta_csv).read_text(encoding=encoding, errors="replace")
    lines = raw.splitlines()  # Este método siempre devuelve una lista
    return lines

# Devuelve la sección "Información Contextual" como un DF
def parse_info_contextual(lines, idx_info: int = 0) -> pd.DataFrame:
    line = lines[idx_info]  # Se toma únicamente la primera línea
    cells = next(csv.reader([line], delimiter=",", quotechar='"'))

    # Limpieza eliminando vacíos y el elemento "Valores reales"
    cleaned = []
    for c in cells:
        c = c.strip().strip('"')
        if not c:  # Vacío -> se omite
            continue
        if c.lower() == "valores reales":
            continue
        cleaned.append(c)

    claves  = cleaned[0::2]
    valores = cleaned[1::2]

    df_info = pd.DataFrame([valores], columns=claves)
    return df_info

# Devuelve la sección "Estadísticos" como un DF
def parse_estadisticos(lines, idx_start: int, idx_end: int) -> pd.DataFrame:

    # Se unen las líneas de la sección en un solo string tipo CSV
    block = "\n".join(lines[idx_start:idx_end])

    # Leemos este bloque como si fuera un CSV independiente
    df_stats = pd.read_csv(
        StringIO(block),
        sep=",",
        decimal=",",
        thousands=".",
        quotechar='"',
        encoding=ENCODING_ALS,
        engine="python",
        )
    # Los índices del df son los nombres de los estadísticos
    df_stats.index.name = "Estadísticos"

    # Ajuste de los encabezados de las columnas
    cols = list(df_stats.columns)
    if str(cols[0]).strip() == "":
      df_stats.columns = cols[1:] + cols[:1] # Se desplazan las columnas a la izq
      df_stats = df_stats.dropna(axis=1, how="all") # Elimina la columna vacía

    return df_stats

def parse_datos_crudos(lines, idx_start: int) -> pd.DataFrame:
    """
    Toma el bloque de 'Datos crudos' desde idx_start hasta el final del archivo
    y lo convierte en DataFrame.
    """
    block = "\n".join(lines[idx_start:])  # desde idx_start hasta EOF

    df_raw = pd.read_csv(
        StringIO(block),
        sep=",",
        decimal=",",
        thousands=".",
        quotechar='"',
        encoding=ENCODING_ALS,
        engine="python",
    )

    return df_raw


In [61]:
# ============================
# Ejecutar: cortar secciones y obtener DataFrames
# ============================

# 1) Leer todas las líneas del archivo
lines = leer_lineas(RUTA_IN, encoding=ENCODING_ALS)
print(f"Número total de líneas en el archivo: {len(lines)}")

# 2) Información contextual
df_info = parse_info_contextual(lines, idx_info=idx_info)
print("Información contextual (DataFrame):")
display(df_info)

# 3) Estadísticos
df_stats = parse_estadisticos(lines, idx_start=idx_stats_start, idx_end=idx_stats_end)
print("Estadísticos (vista rápida):")
display(df_stats.head())

# 4) Datos crudos
df_raw = parse_datos_crudos(lines, idx_start=idx_raw_start)
print("Datos crudos (vista rápida):")
display(df_raw.head())


Número total de líneas en el archivo: 124
Información contextual (DataFrame):


,Máquina,Pedido,Programa
0,INY14,194209,12667-1


Estadísticos (vista rápida):


,t4012 [s],t4018 [s],t4015 [s],V4062 [cmł],p4072 [bar],V4065 [cmł]
Estadísticos,,,,,,
Valor nominal,16.64,0.69,1.85,7.56,1493,9.98
Tolerancia inferior,15.64,0.59,-0.15,6.06,1343,9.48
Tolerancia superior,17.64,0.79,3.85,9.06,1643,10.48
Mínimo,16.89,0.69,1.87,7.45,1536,9.98
Máximo,17.6,0.7,1.94,7.86,1594,10


Datos crudos (vista rápida):


,Muestra aleatoria,Momento,Cantidad piezas,t4012 [s],t4018 [s],t4015 [s],V4062 [cmł],p4072 [bar],V4065 [cmł]
65/1,8 oct 2025 5:31:05,4968,17.37,0.69,1.89,7.67,1563,10,NaN
65/2,8 oct 2025 5:31:22,4976,17.44,0.69,1.88,7.7,1580,9.99,NaN
65/3,8 oct 2025 5:31:40,4984,17.49,0.69,1.88,7.72,1581,9.98,NaN
65/4,8 oct 2025 5:31:57,4992,17.38,0.69,1.89,7.67,1566,9.99,NaN
65/5,8 oct 2025 5:32:15,5000,17.51,0.69,1.88,7.75,1580,10,NaN


In [ ]:
df_stats.columns[0]

In [ ]:
# ============================
# Guardar cada sección en CSV (opcional)
# ============================

df_info.to_csv("ALS_info_contextual.csv", index=False, encoding="utf-8")
df_stats.to_csv("ALS_estadisticos.csv", index=False, encoding="utf-8")
df_raw.to_csv("ALS_datos_crudos.csv", index=False, encoding="utf-8")

print("Archivos generados:")
print(" - ALS_info_contextual.csv")
print(" - ALS_estadisticos.csv")
print(" - ALS_datos_crudos.csv")


**DETECCIÓN Y SEPARACIÓN AUTOMÁTICA DE LAS SECCIONES DEL CSV**

El siguiente código separa en DataFrames cada una de las secciones del CSV de manera automática. Este código es más complejo, pero permite su utilización aún si el archivo CSV de origen cambia su distribución.

**CÓDIGO A UTILIZAR**

In [ ]:
# Importaciones y configuración de display
import csv, re
from pathlib import Path
import pandas as pd

pd.set_option("display.max_columns", 100)  # Muestra hasta 100 columnas sin ocultar intermedias
pd.set_option("display.float_format", lambda v: f"{v:.6g}")  # Formato para mostrar flotantes (6 cifras significativas)


**CÓDIGO COMENTADO**

Aún necesita ajustes para funcionar

In [ ]:
# ============================
# Importaciones y configuración
# ============================

# Módulos estándar
import csv       # lector robusto de CSV respetando comillas y separadores
import re        # expresiones regulares para limpiar encabezados con unidades
from pathlib import Path  # manejo seguro de rutas de archivos

# Terceros
import pandas as pd  # análisis tabular

# Opciones de visualización para que la salida sea clara en Colab
pd.set_option("display.max_columns", 100)                    # mostrar muchas columnas sin truncar
pd.set_option("display.float_format", lambda v: f"{v:.6g}")  # formato compacto de flotantes


In [ ]:
# ============================
# Funciones auxiliares (helpers)
# ============================

def _first_cell_lower(line: str):
    """
    Toma una línea cruda del archivo (string), la parsea como CSV y
    devuelve la PRIMERA celda en minúsculas, o None si la línea está vacía.
    Esto se usa para detectar el encabezado 'Muestra aleatoria' de la sección de datos crudos.
    """
    # csv.reader respeta el separador ',' y las comillas '"'
    cells = next(csv.reader([line], delimiter=",", quotechar='"'))
    if not cells:               # línea vacía -> no hay celdas
        return None
    # quitamos espacios y comillas residuales, y pasamos a minúsculas
    return cells[0].strip().strip('"').lower()


def _clean_param_header(name: str) -> str:
    """
    Limpia el nombre de una columna de parámetro:
    - elimina unidades entre corchetes (ej. 't4012 [s]' -> 't4012')
    - remueve espacios sobrantes
    """
    # patrón: cualquier cosa entre corchetes [ ... ] con espacios opcionales alrededor
    name = re.sub(r"\s*\[.*?\]\s*", "", str(name))
    return name.strip()


def _find_section_indices(lines):
    """
    Localiza los índices (número de línea) de las cabeceras de:
      - 'Estadísticos' (idx_stats_header): línea donde están los nombres de parámetros (t4012, t4018, ...)
      - 'Datos crudos' (idx_raw_header): línea cuya primera celda dice 'Muestra aleatoria'
    Devuelve una tupla (idx_stats_header, idx_raw_header). Si alguno no se encuentra, deja None.
    """
    idx_stats_header = None

    # Escaneamos las primeras ~120 líneas buscando una cabecera que contenga varias claves típicas
    for i, ln in enumerate(lines[:120]):
        cells = next(csv.reader([ln], delimiter=",", quotechar='"'))
        if not cells:
            continue  # línea en blanco
        joined = ",".join(cells).lower()
        # Heurística: deben aparecer varios parámetros conocidos en la línea de cabecera
        if all(k in joined for k in ["t4012", "t4018", "v4065"]):
            idx_stats_header = i
            break

    idx_raw_header = None
    # Recorremos todas las líneas para encontrar la cabecera de 'Datos crudos'
    for i, ln in enumerate(lines):
        fc = _first_cell_lower(ln)
        if fc and fc.startswith("muestra aleatoria"):
            idx_raw_header = i
            break

    return idx_stats_header, idx_raw_header


In [ ]:
# ============================
# Núcleo: lectura + construcción del resumen
# ============================

def leer_estadisticos(ruta_csv: str) -> pd.DataFrame:
    """
    Lee el CSV del ALS y retorna el bloque 'Estadísticos' como DataFrame.
    Maneja el formato específico del ALS:
      - separador de campo: ','
      - valores numéricos entre comillas '"..."'
      - coma como separador decimal: decimal=','
      - punto como separador de miles: thousands='.'
      - codificación latin1 (acentos, caracteres especiales)
    """
    # Leemos el archivo completo como texto para poder detectar secciones por líneas
    p = Path(ruta_csv)
    raw = p.read_text(encoding="latin1", errors="replace")
    lines = raw.splitlines()  # lista de líneas del archivo

    # Detectamos índices de inicio de 'Estadísticos' y 'Datos crudos'
    idx_stats_header, idx_raw_header = _find_section_indices(lines)
    if idx_stats_header is None or idx_raw_header is None:
        raise RuntimeError(
            f"No se encontraron cabeceras (stats={idx_stats_header}, raw={idx_raw_header})."
        )

    # Número de filas de la tabla de 'Estadísticos' (excluyendo la cabecera de crudos)
    nrows_stats = idx_raw_header - idx_stats_header - 1
    if nrows_stats <= 0:
        raise RuntimeError("Rango de 'Estadísticos' inválido (nrows_stats <= 0).")

    # Leemos solo el bloque de 'Estadísticos' con pandas.read_csv
    df_stats = pd.read_csv(
        ruta_csv,
        skiprows=idx_stats_header,  # salta líneas anteriores y deja la cabecera de parámetros como header=0
        nrows=nrows_stats,          # limita la lectura hasta antes de los datos crudos
        header=0,
        sep=",",                    # separador de campo
        decimal=",",                # coma decimal
        thousands=".",              # punto de miles
        quotechar='"',              # valores numéricos entre comillas
        encoding="latin1",
        engine="python",            # motor más tolerante con formatos “no estándar”
    )

    # Normalizamos el nombre de la primera columna, que contiene el nombre del estadístico
    first_col = df_stats.columns[0]
    df_stats.rename(columns={first_col: "Estadístico"}, inplace=True)

    # Limpiamos las unidades de los encabezados de parámetros (t4012 [s] -> t4012)
    df_stats.columns = ["Estadístico"] + [_clean_param_header(c) for c in df_stats.columns[1:]]

    return df_stats


def construir_tabla_resumen(df_stats: pd.DataFrame) -> pd.DataFrame:
    """
    Devuelve un DataFrame solo con estas filas, en este orden:
      - Valor nominal
      - Tolerancia inferior
      - Tolerancia superior
      - xqq
      - Sigma
      - Cp
      - Cpd
    Convierte las columnas de parámetros a numérico cuando es posible.
    """
    # Lista objetivo en el orden que se desea reportar
    requeridos = [
        "Valor nominal",
        "Tolerancia inferior",
        "Tolerancia superior",
        "xqq",
        "Sigma",
        "Cp",
        "Cpd",
    ]

    # Creamos una columna auxiliar en minúsculas para comparar sin problemas de mayúsculas/espacios
    df_aux = df_stats.copy()
    df_aux["__norm__"] = df_aux["Estadístico"].astype(str).str.strip().str.lower()

    # Acumulamos las filas encontradas respetando el orden requerido
    out = []
    for etiqueta in requeridos:
        fila = df_aux[df_aux["__norm__"] == etiqueta.lower()]
        if not fila.empty:
            out.append(fila.drop(columns="__norm__"))

    if not out:
        # Si no se encontró nada, algo no coincide con el formato/etiquetas esperadas
        raise RuntimeError("No se hallaron las filas requeridas en 'Estadísticos'.")

    # Unimos todas las filas y dejamos 'Estadístico' como índice
    resumen = pd.concat(out, ignore_index=True).set_index("Estadístico")

    # Convertimos todas las columnas (parámetros) a numérico cuando aplique
    for c in resumen.columns:
        resumen[c] = pd.to_numeric(resumen[c], errors="coerce")

    return resumen


In [ ]:
# ============================
# Subida del archivo CSV (Colab)
# ============================

from google.colab import files

# Abre el cuadro de diálogo para seleccionar archivos
uploaded = files.upload()

# Tomamos el primer archivo subido y guardamos su nombre en RUTA_IN
RUTA_IN = next(iter(uploaded.keys()))
print("Archivo cargado:", RUTA_IN)


In [ ]:
# ============================
# Ejecución: lectura y resumen de 'Estadísticos'
# ============================

# 1) Leer bloque de 'Estadísticos'
df_stats = leer_estadisticos(RUTA_IN)

# 2) Construir la tabla solicitada
resumen = construir_tabla_resumen(df_stats)

# 3) Mostrar resultados en pantalla
print("Bloque 'Estadísticos' (vista rápida):")
display(df_stats.head(10))

print("Resumen solicitado (Valor nominal, Tol. inf., Tol. sup., xqq, Sigma, Cp, Cpd):")
display(resumen)

# 4) (Opcional) Guardar a CSV el resumen
OUT = "ALS_Estadisticos_Resumen.csv"
resumen.to_csv(OUT, encoding="utf-8")
print("Guardado:", OUT)


In [ ]:
# ============================
# Validaciones rápidas (opcional)
# ============================

# Validar que las columnas de parámetros esperadas estén presentes en 'Estadísticos'
cols_ok = {"t4012", "t4018", "t4015", "v4062", "p4072", "v4065"}
presentes = {c.lower() for c in df_stats.columns}
assert cols_ok.issubset(presentes), f"Faltan columnas en 'Estadísticos': {cols_ok - presentes}"

# Validar que las filas requeridas estén en el resumen
esperados = {"Valor nominal","Tolerancia inferior","Tolerancia superior","xqq","Sigma","Cp","Cpd"}
faltantes = esperados - set(resumen.index)
if faltantes:
    print("⚠️ Faltan estadísticos en el resumen:", faltantes)
else:
    print("✔️ Resumen completo y consistente.")
